In [9]:
from __future__ import print_function
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import AveragePooling2D, Input, Flatten, Activation, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras import backend as K
from keras.models import Sequential
import pandas as pd
from os import path
from shutil import copy
import random
import numpy as np
import CopyFiles

/home/ivana/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
def sve_jpg(df):
    for item in list(df.new_filename):
        if not '.jpg' in item:
            return False
    return True

In [8]:
import os
from shutil import copy
import pandas as pd
df = pd.read_csv('/home/ivana/repos/Vje-ba/all_data_info.csv')
seed = 123

print(df.shape)
df.head()

(103250, 12)


,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [6]:
print('Sve su jpg: ' + str(sve_jpg(df)))

Sve su jpg: True


In [10]:
x = list(df['artist'].value_counts())
# broj umjetnika koji imaju vise ili jednako od 300 slika
print(len([a for a in x if a >= 300]))
# len(set(x)) #---> ukupan broj umjetnika

57


In [163]:
# train, validation, test --- 80, 10, 10
num_train = 240
num_val = 30
num_test = num_val
num_samples = num_train + num_val + num_test
b_size = 60

#lista umjetnika koje ćemo promatrati
temp = df['artist'].value_counts()
threshold = num_samples
artists = temp[temp >= threshold].index.tolist()
# print(artists)

num_artists = len(artists)
print('Prepoznajemo ' + str(num_artist) + ' umjetnika')

train_dfs = []
val_dfs = []
test_dfs = []

for a in artists:
    # PROVJERI KASNIJE ŠTA JE S NA=TRUE
    tmp = df[df['artist'].str.startswith(a)].sample(n=num_samples, random_state=seed)
    # print(tmp.shape)
    t_df = tmp.sample(n=num_train, random_state=seed)
    rest_df = tmp.loc[~tmp.index.isin(t_df.index)] # uzmi komplement od t_df
    # print(rest_df.shape)
    v_df = rest_df.sample(n=num_val, random_state=seed)
    te_df = rest_df.loc[~rest_df.index.isin(v_df.index)]
    
    train_dfs.append(t_df)
    val_dfs.append(v_df)
    test_dfs.append(te_df)
    
    # ovo se pokrene samo jednom!!
    copyImagesToFiles(a, t_df, v_df, te_df)

train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)
test_df = pd.concat(test_dfs)

print('train tablica\t\t', train_df.shape)
print('validation tablica\t', val_df.shape)
print('test tablica\t\t', test_df.shape)

['Pierre-Auguste Renoir', 'Marc Chagall', 'Zdislav Beksinski', 'John Singer Sargent', 'Giovanni Battista Piranesi', 'Ivan Aivazovsky', 'Ivan Shishkin', 'Rembrandt', 'Pablo Picasso', 'Gustave Dore', 'Albrecht Durer', 'Ilya Repin']
12


In [135]:
def center_crop(img, center_crop_size):
    assert img.shape[2] == 3
    centerw, centerh = img.shape[0] // 2, img.shape[1] // 2
    halfw, halfh = center_crop_size[0] // 2, center_crop_size[1] // 2
    return img[centerw-halfw:centerw+halfw, centerh-halfh:centerh+halfh, :]

# https://jkjung-avt.github.io/keras-image-cropping/
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length, random_crop=True):
    '''
    Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator
    '''
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            if random_crop == True:
                batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
            else:
                batch_crops[i] = center_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)


In [160]:
# velicina slika koje dajemo ulaznom sloju mreze
input_shape = (224, 224, 3)
# velicina batch-a
b_size = 87

train_datagen = ImageDataGenerator(
                horizontal_flip=True)

val_datagen = ImageDataGenerator(
                horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
                    '../train',
                    batch_size=b_size,
                    class_mode='sparse')
# na slikama iz train skupa radimo crop na slučajnom mjestu
train_crops = crop_generator(train_generator, 224)

validation_generator = val_datagen.flow_from_directory(
                    '../validation',
                    batch_size=b_size,
                    class_mode='sparse')
# na slikama iz validation skupa radimo centralni crop
val_crops = crop_generator(validation_generator, 224, False)

x2 (1, 800, 591, 3)
(1, 224, 224, 3)
(1, 800, 591, 3)
(224, 224, 3)


Model mreže

In [ ]:
# model mreže inspiriran glavnim člankom

model = Sequential()
#nn.Conv2d(3, 32, kernel_size=3 +, stride=2 +, padding=1), # -> 112
model.add(Conv2D(32, kernel_size=3, strides=2, padding="same",input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, kernel_size=3, strides=2, padding="same", input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(4*num_artists, input_shape=(6272,))) #nn.Linear(6272, 4*num_artists),
model.add(Activation('relu'))
model.add(Dense(num_artists, input_shape=(4*num_artists,))) #nn.Linear(4*num_artists, num_artists)

# koristimo adamov optimizator i metrika je točnost
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

# crta tablicu slojeva mreže
model.summary()

In [ ]:
# treniramo mrežu....

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

print('STEP_SIZE_TRAIN', STEP_SIZE_TRAIN)
print('STEP_SIZE_VALID', STEP_SIZE_VALID)

model.fit_generator(
    train_crops,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=3,
    validation_data=val_crops,
    validation_steps=STEP_SIZE_VALID)

model.save_weights('prvi_pokusaj300SVE-SLIKE.h5')

In [ ]:
# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])